# Text Generation

Mainly copied and adapted from the Text Generation RNN example

In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#import dataset
path_to_file = "corpus.txt"


In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1334722 characters


In [4]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

89 unique characters


## Text Vectorisation

In [5]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [6]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

In [7]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'                    DAVID ATTENBOROUGH: The natural world is full  of extraordinarily shaped creature'
's,  but how have the stretched bodies of some given them an edge?  I have had the fortune to meet  so'
"me of the planet's most enchanting creatures, but some  stand out more than others, because of their "
'intriguing biology.\n  Our knowledge of some of these creatures extends back centuries.\n  Others we ha'
've discovered more recently.\n  In this series, I share their stories  and reveal why they are conside'


In [8]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

## Training Batches

In [9]:
# Batch size 
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build Model

In [10]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [11]:
if tf.test.is_gpu_available(): #GPU
  rnn = tf.keras.layers.CuDNNGRU
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [12]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
    # tack on two RNN layers 
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
    
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
      
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [13]:
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

## Try Model

In [14]:
for input_example_batch, target_example_batch in dataset.take(1): 
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 89) # (batch_size, sequence_length, vocab_size)


In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           22784     
_________________________________________________________________
cu_dnngru (CuDNNGRU)         (64, None, 1024)          3938304   
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (64, None, 1024)          6297600   
_________________________________________________________________
cu_dnngru_2 (CuDNNGRU)       (64, None, 1024)          6297600   
_________________________________________________________________
dense (Dense)                (64, None, 89)            91225     
Total params: 16,647,513
Trainable params: 16,647,513
Non-trainable params: 0
_________________________________________________________________


## Train

In [16]:
def loss(labels, logits):
#   return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
  return tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 89)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.4890766


In [17]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

Checkpoints

In [18]:
# Directory where the checkpoints will be saved
checkpoint_dir = './text100'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [19]:
EPOCHS=45

In [20]:
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/45
208/208 [==============================] - 31s 148ms/step - loss: 2.3123
Epoch 2/45
208/208 [==============================] - 30s 146ms/step - loss: 1.4729
Epoch 3/45
208/208 [==============================] - 29s 141ms/step - loss: 1.2796
Epoch 4/45
208/208 [==============================] - 30s 144ms/step - loss: 1.1967
Epoch 5/45
208/208 [==============================] - 30s 143ms/step - loss: 1.1388
Epoch 6/45
208/208 [==============================] - 29s 142ms/step - loss: 1.0914
Epoch 7/45
208/208 [==============================] - 30s 145ms/step - loss: 1.0492
Epoch 8/45
208/208 [==============================] - 29s 140ms/step - loss: 1.0104
Epoch 9/45
208/208 [==============================] - 30s 144ms/step - loss: 0.9709
Epoch 10/45
208/208 [==============================] - 29s 138ms/step - loss: 0.9343
Epoch 11/45
208/208 [==============================] - 29s 141ms/step - loss: 0.8975
Epoch 12/45
208/208 [==============================] - 30s 146ms/step - lo

## Generate

In [21]:
tf.train.latest_checkpoint(checkpoint_dir)

'./text100/ckpt_45'

In [22]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1) 

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [23]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            22784     
_________________________________________________________________
cu_dnngru_3 (CuDNNGRU)       (1, None, 1024)           3938304   
_________________________________________________________________
cu_dnngru_4 (CuDNNGRU)       (1, None, 1024)           6297600   
_________________________________________________________________
cu_dnngru_5 (CuDNNGRU)       (1, None, 1024)           6297600   
_________________________________________________________________
dense_1 (Dense)              (1, None, 89)             91225     
Total params: 16,647,513
Trainable params: 16,647,513
Non-trainable params: 0
_________________________________________________________________


In [51]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 4500

  # Converting our start string to numbers (vectorizing) 
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  temperature = 0.7 #reduce temperature to 0.7 non words appear at higher temperatures 

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      
      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [52]:
# print(generate_text(model, start_string=u"ROMEO"))
print(generate_text(model, start_string=u"ATTENBOROUGH:"))


ATTENBOROUGH: The power of the sun  drives the seasons, transformed our image and understandably out of here.
  They are very rare only a few days,  but it doesn't make it through the sponge's walls.
  His run itself up here and then release them.
  He was hardly be new life.
  He's a bit of a two-year-old son, Tatu can't grab them.
  So, why both parents have to tackle the difficulties of life  deeper into the water.
  It's a calf to succeed here,  but another week planted pouch of our pain.
  It's very rare for another two years.
  The Austrian one lasted just a few days.
  Until now, at last, tell the rest combined.
  Unlike made from the brine, super-pods breen,  the length of a butterfly's life cycle,  or the strange biology of the emperor penguin.
  Some of these creatures were surrounded by myths and misunderstanding of forest  and the cubs' cluckications in particular type of sperm  and they've now disappeared with death.
  But carotene is not only seaweed on which to live.
  A